# Config & Imports

In [1]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

import os
import tensorflow as tf
from tensorflow import keras
import keras.layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.callbacks import EarlyStopping

from keras.regularizers import L1L2
from keras.regularizers import L2

import utilities_LR

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

import sys

In [2]:
config = {
    'data': {
        'n_datasets': 45_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 10, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        ### int or 'random'
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 1.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 44,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False, # should be always false
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae',# keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 500,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True, # usually true
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.15,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            },
            'train_noise': 0 # y_flip fraction on Y_train pred data from lambda net
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 28,
        'use_gpu': True,
        'gpu_numbers': '5',
        'RANDOM_SEED': 1,   
    }
}

In [3]:
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_numbers'] if config['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

In [5]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  1
Num XLA-GPUs Available:  0


# Functions

In [6]:
def load_lambda():
    directory = utilities_LR.lambda_path_LR(config)
    
    return np.load(directory + '/lambda_weights_list.npy', allow_pickle=True)

In [7]:
def get_X_data():
    directory = utilities_LR.data_path_LR(config)
    return np.load(directory + '/X.npy', allow_pickle=True)

In [8]:
def get_y_pred_data():
    directory = utilities_LR.lambda_path_LR(config)
    
    return np.load(directory + '/lambda_preds_list.npy', allow_pickle=True)
    #return np.load(directory + '/lambda_preds_list_with0.2_noise.npy', allow_pickle=True)

# Model Training

In [9]:
lambda_nets = load_lambda()

In [10]:
y_predictions = get_y_pred_data()

In [11]:
valid_feature_data = get_X_data()

In [12]:
y_predictions = y_predictions.round()

In [13]:
indexcol = np.array([np.arange(config['data']['n_datasets'], dtype=int)], ndmin=2).reshape(config['data']['n_datasets'], 1)
y_predictions = np.concatenate([indexcol, y_predictions], axis=1)

In [14]:
import pandas as pd

In [15]:
lambda_nets

array([[ 1.3550539e+00,  1.2586989e+00,  1.2124062e+00, ...,
        -4.2458245e-01,  4.9477452e-01,  2.0274468e-02],
       [ 1.1479834e+00,  9.6538615e-01,  1.2207832e+00, ...,
         4.2061946e-01,  3.3673701e-01,  3.1846145e-04],
       [ 1.3644235e+00,  1.7095858e+00,  1.0799276e+00, ...,
        -2.9329687e-01, -3.2810420e-01,  8.7037854e-02],
       ...,
       [ 1.3203987e+00,  1.3999584e+00,  1.4361682e+00, ...,
         4.1442773e-01,  7.5010920e-01,  1.5540089e-01],
       [ 1.1476049e+00,  1.9412074e+00,  1.2327747e+00, ...,
        -5.0742167e-01,  5.8738905e-01,  9.2185505e-02],
       [ 1.3177347e+00,  1.1020919e+00,  1.3079326e+00, ...,
        -4.3530583e-01, -7.9997015e-01,  8.3676837e-02]], dtype=float32)

In [16]:
lambda_nets.shape

(45000, 641)

In [17]:
y_predictions.shape

(45000, 5001)

In [18]:
valid_feature_data = tf.convert_to_tensor(valid_feature_data, dtype=tf.float32)

# Custom Loss

In [19]:
@tf.function
def custom_loss(y_predictions_index, y_coef_pred):
    
    index = y_predictions_index[:, 0]
    y_true = y_predictions_index[:, 1:]
    
    # ADD NOISE TO TRAINING DATA
    noise = tf.cast(config['inets']['data_prep']['train_noise'], tf.float32)
    noise_logits = tf.tile([[1-noise, noise]], [tf.shape(y_true)[0], tf.constant(1)])
    noise_flip = tf.random.categorical(tf.math.log(noise_logits), y_true.shape[1])
    y_true = y_true*(1-noise_flip) + (1-y_true)*noise_flip   
    
    index = tf.cast(index, tf.int32)
    
    X_feature_data_samples = tf.gather(valid_feature_data, index)
    
    y_pred = tf.transpose(tf.math.add(tf.transpose(tf.linalg.matvec(X_feature_data_samples, y_coef_pred[:, 1:])), y_coef_pred[:, 0]))

    metric = tf.keras.losses.BinaryCrossentropy(
                                from_logits=True,
                                label_smoothing=0.0,
                                axis=-1,
                                reduction='auto',
                                name='binary_crossentropy')
    loss = metric(y_true, y_pred)
    return loss

In [20]:
# Data Prep

early_stopping = EarlyStopping(monitor='val_loss',
                                min_delta=0.001,
                                patience=45,
                                verbose=0,
                                mode='auto',
                                baseline=None,
                                restore_best_weights=True)


#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
#                              patience=5, min_lr=0.0001)
#

#X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
#                                                                      y)

X_train, X_test, y_train, y_test = train_test_split(lambda_nets, 
                                                    y_predictions, 
                                                    test_size=config['inets']['data_prep']['train_test_val_split']['test_size'], 
                                                    random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])




# Model Def
model = Sequential()
#model.add(BatchNormalization())
#regval = 0.0000000000001
#model.add(Dense(2056, activation='swish', kernel_regularizer=L2(regval), bias_regularizer= L2(regval), input_dim=X_train.shape[1]))
##model.add(keras.layers.Dropout(0.1))
#model.add(Dense(1028, activation='swish', kernel_regularizer=L2(regval), bias_regularizer= L2(regval)))
##model.add(keras.layers.Dropout(0.2))
#model.add(Dense(1028, activation='swish', kernel_regularizer=L2(regval), bias_regularizer= L2(regval)))
##model.add(keras.layers.Dropout(0.2))
#model.add(Dense(1028, activation='swish', kernel_regularizer=L2(regval), bias_regularizer=  L2(regval)))
##model.add(keras.layers.Dropout(0.1))

#model.add(Dense(10000, activation='ELU', input_dim=X_train.shape[1]))
##model.add(keras.layers.Dropout(0.3))
##model.add(Dense(4112, activation='relu'))
#model.add(Dense(10000, activation='ELU'))
#model.add(Dense(10000, activation='ELU'))
#model.add(Dense(10000, activation='ELU'))
#model.add(Dense(10000, activation='ELU'))
#model.add(Dense(10000, activation='ELU'))
#model.add(Dense(10000, activation='ELU'))

#model.add(Dense(1028, activation='LeakyReLU', kernel_regularizer=L2(regval), bias_regularizer= L2(regval), input_dim=X_train.shape[1]))
##model.add(keras.layers.Dropout(0.2))
#model.add(Dense(1028, activation='LeakyReLU', kernel_regularizer=L2(regval), bias_regularizer= L2(regval)))
##model.add(keras.layers.Dropout(0.2))
#model.add(Dense(2000, activation='LeakyReLU', kernel_regularizer=L2(regval), bias_regularizer=  L2(regval), input_dim=X_train.shape[1]))
#model.add(keras.layers.Dropout(0.1))


#model.add(Dense(4096, activation='swish'))
#model.add(Dense(2048, activation='swish'))
#model.add(Dense(1024, activation='swish'))
#model.add(Dense(512 , activation='swish'))

model.add(Dense(2048, activation='swish'))
model.add(keras.layers.Dropout(0.5))

model.add(Dense(1024, activation='swish'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(512 , activation='swish'))
model.add(keras.layers.Dropout(0.5))

model.add(Dense(config['data']['n_features'] + 1, activation='linear'))

#lr = tf.keras.optimizers.schedules.CosineDecayRestarts(
#                                                        0.01,
#                                                        10,
#                                                        t_mul=2.0,
#                                                        m_mul=1.0,
#                                                        alpha=0.0,
#                                                        name=None
#)
#
#
#opt = tf.keras.optimizers.RMSprop(
#    #learning_rate=0.001,
#    learning_rate=lr,
#    rho=0.9,
#    momentum=0.0,
#    epsilon=1e-07,
#    centered=False,
#    name="RMSprop",
#)

opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999, # 0.999
    epsilon=1e-08, # 1.0, 0.1, 0.01
    amsgrad=False,
    name='Adam',)

#opt = tf.keras.optimizers.SGD(
#    learning_rate=0.1, # 0.001
#    momentum=0.0,
#    nesterov=False,
#    name='SGD'
#)

model.compile(optimizer=opt, loss=custom_loss, metrics=[])
model.build(input_shape=X_train.shape)
model.summary()

# Model fit
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=config['inets']['model_fit']['batch_size'],
                    epochs=config['inets']['model_fit']['epochs'],
                    verbose=config['inets']['model_fit']['verbose'],
                    callbacks=[early_stopping],
                    validation_split=config['inets']['data_prep']['train_test_val_split']['val_size'],
                    shuffle=config['inets']['model_fit']['shuffle'],
                    class_weight=config['inets']['model_fit']['class_weight'],
                    sample_weight=config['inets']['model_fit']['sample_weight'],
                    initial_epoch=config['inets']['model_fit']['initial_epoch'],
                    steps_per_epoch=config['inets']['model_fit']['steps_per_epoch'],
                    validation_steps=config['inets']['model_fit']['validation_steps'],
                    validation_batch_size=config['inets']['model_fit']['validation_batch_size'],
                    validation_freq=config['inets']['model_fit']['validation_freq'],
                    workers=config['computation']['n_jobs'],
                    #use_multiprocessing=True
                   )
print(history.history['val_loss'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (40500, 2048)             1314816   
                                                                 
 dropout (Dropout)           (40500, 2048)             0         
                                                                 
 dense_1 (Dense)             (40500, 1024)             2098176   
                                                                 
 dropout_1 (Dropout)         (40500, 1024)             0         
                                                                 
 dense_2 (Dense)             (40500, 512)              524800    
                                                                 
 dropout_2 (Dropout)         (40500, 512)              0         
                                                                 
 dense_3 (Dense)             (40500, 11)               5

In [21]:
def save_model(model):
    path = utilities_LR.inet_path_LR(config)
    
    Path(path).mkdir(parents=True, exist_ok=True)
    model.save(path + '/modelKeras' + '.h5')

In [22]:
save_model(model)

In [23]:
def save_testData(X_test, y_test):
    path = utilities_LR.inet_path_LR(config)
    
    Path(path).mkdir(parents=True, exist_ok=True)
    np.save(path + '/X_test', X_test)
    
    np.save(path + '/y_test', y_test)

In [24]:
save_testData(X_test, y_test)